In [1]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
warnings.filterwarnings("ignore")


In [15]:
def getUniqueModelsForTicker(tickerToTrade):
    allModels = portfolio.getModels(tickerToTrade, returnEntireObject=True)
    print("***ORIGINAL MODELS***")
    for modelEntity in allModels:
        model = modelEntity["model"]
        print(model.describe())
    print("***UNIQUE MODELS***")
    uniqueModels = portfolioGeneration.getUniqueModels(allModels)
    for model in uniqueModels:
        print(model.describe())
    return uniqueModels

In [16]:
tickerToTrade = "EWT"

In [17]:
uniqueModels = getUniqueModelsForTicker(tickerToTrade)

***ORIGINAL MODELS***
((10, ('IEF', 3, None, None), 3, 'EWT'), 10, 250)
((10, ('EWH', 5, None, 10), 3, 'EWT'), 10, 100)
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25)
((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50)
***UNIQUE MODELS***
((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50)
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25)


In [18]:
def computeReturnsForUniqueModels(uniqueModels, factorToTrade):
    tickersRequired = []
    tickersTraded = []
    for mod in uniqueModels:

        print(mod.describe())
        if mod.inputSeries.targetTicker not in tickersRequired:
            tickersRequired.append(mod.inputSeries.targetTicker)
        if mod.inputSeries.series.ticker not in tickersRequired:
            tickersRequired.append(mod.inputSeries.series.ticker)
        if mod.inputSeries.targetTicker not in tickersTraded:
            tickersTraded.append(mod.inputSeries.targetTicker)


    pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

    joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
    modelReturns, modelPredictions = portfolioGeneration.generateAllReturns(uniqueModels, joinedData)
    
    return modelReturns, modelPredictions, modelReturns.join(dataAck.getDailyFactorReturn(factorToTrade, joinedData))

In [19]:
factorToTrade = "SPY"
modelReturns, modelPredictions, modelReturnsWithFactor =\
    computeReturnsForUniqueModels(uniqueModels, factorToTrade)

((10, ('IEF', 3, None, None), 2, 'EWT'), 10, 50)
((10, ('EWH', 5, None, 10), 2, 'EWT'), 10, 25)


Process ForkProcess-4:
Process ForkProcess-3:
Process ForkProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py", line 31, in plainSources
    thisData = getTickerData(tick)
Traceback (most recent call last):
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py", line 31, in plainSources
    thisData = getTickerData(tick)
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py", line 7, in getTickerData
    limitedDf = quandl.get(["EOD/" + ticker +".11"], authtoken="G3AvFe4ZUZoBEthhjmEY")
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py", line 7, in getTickerData
    limitedDf = quandl.get(["EOD/" + ticker +".11"], authtoken="G3AvFe4ZUZoBEthhjmEY")
  File "/Users/patrickogrady/Documents/CODE/walkforwardTrader/dataAck.py", line 31, in plainSources
    thisData = getTickerData(tick)
  File "/Users/patrickogrady/anaconda3/lib/pyth

KeyboardInterrupt: 

  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/pandas/core/tools/datetimes.py", line 449, in to_datetime
    if arg is None:
KeyboardInterrupt
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/quandl/model/merged_dataset.py", line 73, in _get_dataset_data
    return dataset.data(**updated_options)
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/quandl/model/dataset.py", line 47, in data
    return Data.all(**updated_options)

During handling of the above exception, another exception occurred:

  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/quandl/model/dataset.py", line 47, in data
    return Data.all(**updated_options)
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/quandl/operations/list.py", line 16, in all
    Util.convert_to_dates(response_data)
  File "/Users/patrickogrady/anaconda3/lib/python3.6/site-packages/quandl/operations/list.py", line 16, in all
    Util.convert_to_dates(respons

In [ ]:
%matplotlib inline
portfolioGeneration.visualizeModels(modelReturnsWithFactor)

In [ ]:
portfolioGeneration.storePastPredictions(uniqueModels, modelPredictions)

In [ ]:
portfolioHash = portfolioGeneration.storePortfolio(uniqueModels,\
            description="Just trade underlying", benchmark=factorToTrade)

In [ ]:
modelHashes = portfolioGeneration.getPortfolioModels(portfolioHash)

In [ ]:
allModels = portfolio.getModelsByKey(modelHashes)

In [ ]:
joinedData = potfolioGeneration.getPertinentDataForModels(allModels)

In [ ]:
portfolioGeneration.generateRawPredictions(allModels, joinedData, daysBack = True)

In [ ]:
aggregateReturns, aggregatePredictions = portfolioGeneration.generateAggregateReturnsPredictions(allModels, joinedData)

In [ ]:
hrpReturns, historicalWeights = portfolioGeneration.produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in allModels]], 22, startIndex=None, maxWindowSize=True)
print("COMPUTED HISTORICAL WEIGHTS")
portfolioGeneration.storeHistoricalAllocations(portfolioHash, allModels, historicalWeights, aggregatePredictions)

In [ ]:
portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
portfolioInfo = {
    "key":portfolioInfo.key.name,
    "description":portfolioInfo["description"],
    "benchmark":portfolioInfo["benchmark"]
}
print(portfolioInfo)
portfolioData = portfolioGeneration.getDataForPortfolio(portfolioHash)
portfolioGeneration.cachePortfolio(portfolioInfo, portfolioData)